In [1]:
start_day="2020-3-28"
time_horizon=3
path_to_trend_data = "../../data/iva_kumulativ.csv"
path_to_static_region_data = "../../data/regions.csv"
path_to_static_geojson = "../../data/geocounties.geojson"
solve_time_limit = 60 #seconds
w_total_undercapacity = 100
w_max_under = 100
w_max_over = 1
w_nb_patient_transfers = 1
w_km_patient_transfers = 0.01
w_nb_long_transfers = 0.01
    
# --------------------------------------------------------------------------- #
# Set-up
# --------------------------------------------------------------------------- #  
import warnings
warnings.filterwarnings("ignore")

from datetime import timedelta
from datetime import datetime

today = datetime.strptime(start_day, '%Y-%m-%d')
target_day = today + timedelta(days=time_horizon)

# --------------------------------------------------------------------------- #
# Read data
# --------------------------------------------------------------------------- #  
import read_data_for_optimization 
current_df, trend_dict = read_data_for_optimization.read_and_process_data(trend_data = path_to_trend_data,
                                                                          region_data = path_to_static_region_data,
                                                                          today = today)
initial_map = read_data_for_optimization.plot_initial_state(current_df,geojson=path_to_static_geojson)    

In [16]:
# --------------------------------------------------------------------------- #
# Build and prepare optimization model
# --------------------------------------------------------------------------- # 
import pulp as plp
import build_optimization_model
mdl = build_optimization_model.build_model()
mdl = build_optimization_model.define_model_parameters_and_sets(mdl, current_df)
mdl = build_optimization_model.define_model_variables(mdl)
mdl = build_optimization_model.calculate_distances(mdl, current_df)

import define_model_constraints
mdl = define_model_constraints.exempt_departments(mdl)
mdl = define_model_constraints.set_initial_state(mdl,current_df)
mdl = define_model_constraints.link_x_and_y_vars(mdl)
mdl = define_model_constraints.restrict_transfers(mdl)
mdl = define_model_constraints.transfer_bounds(mdl)
mdl = define_model_constraints.maximum_long_transfers(mdl)
mdl = define_model_constraints.short_transfers_per_dep(mdl)
mdl = define_model_constraints.update_for_next_period(mdl,trend_dict,today,target_day)

import define_model_objective
mdl = define_model_objective.define_total_undercapacity(mdl,current_df)
mdl = define_model_objective.define_distance_measures(mdl)
mdl = define_model_objective.summarize_objectives2(mdl,
                                                  w_total_undercapacity = w_total_undercapacity,
                                                  w_nb_patient_transfers = w_nb_patient_transfers,
                                                  w_km_patient_transfers = w_km_patient_transfers,
                                                  w_nb_long_transfers = w_nb_long_transfers)

In [13]:
#mdl.constraints["use_link_Stockholm_Värmland_0"]
#mdl.constraints["new_cases_Västmanland_0"]
#mdl.constraints.keys()

In [18]:
from pulp import CPLEX
from pulp import GLPK
mdl.solve(solver=CPLEX())

CPXPARAM_Read_DataCheck                          1
Found incumbent of value 1345.621355 after 0.00 sec. (0.07 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 606 rows and 144 columns.
Reduced MIP has 363 rows, 780 columns, and 2298 nonzeros.
Reduced MIP has 318 binaries, 462 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.68 ticks)
Probing time = 0.00 sec. (0.20 ticks)
Tried aggregator 1 time.
Reduced MIP has 363 rows, 780 columns, and 2298 nonzeros.
Reduced MIP has 318 binaries, 462 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.56 ticks)
Probing time = 0.00 sec. (0.19 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.01 sec. (3.50 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                         1

-2